# Bayesian Filtering for Tracking #

## Robot Localization ##

#### Localization is one of the fundamental problems in mobile robotics. There are multiple instances of the localization problem with different difficulties. In this example, we consider a scenario where the robot has a map of the environment. Then we consider two cases: ###

__1. The initial position is known and then needs to keep track of its position.__

__2. Localize itself from scratch when it could be anywhere.__

#### Why not GPS? Because it is not accurate enough in many scenarios. <br> <br> Self-driving cars need accuracy of a few centimeters. Thus they need additional sensors to help with the localization. ####

### Import libraries ###

In [ ]:
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt

## Consider a hallway (ciricular) with door ##
### There is a robot with a sensor inside ###

<img src="circular-hallway-with.jpg" alt="Drawing" style="width: 400px;"/>

<img src="wheel.png" alt="Drawing" style="width: 400px;"/>

<tr>
    <td> <img src="wheel.png" alt="Drawing" style="width: 350px;"/> </td>
    <td> <img src="roger.jpg" alt="Drawing" style="width: 350px;"/> </td>
    </tr>

#### Consider a hallway with three doors and seven wall segments. #### 

In [ ]:
hallway = np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0])

### State ###
#### We consider discrete time. At any time $t$, we  characterize a dynamic system by a state vector $x_t$, also called simply "the state". <br> State of a moving robot on a plane: $x_t=(X_t,Y_t,X^{'}_t,Y^{'}_t)$ where $X_t, Y_t$ refers to the robot’s current position and $X^{'}_t, Y^{'}_t$ to its speed in the X and Y direction, respectively. 

### Measurements
#### The state $x_t$ may be unobservable: it cannot be measured directly. Instead, sensors may generate (noisy) measurements, $e_t$, of phenomena that are caused by the state.

### Belief
#### Since the state $x_t$ is unobservable, we maintain a belief about the state


In [ ]:
belief = np.array([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1])
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)

### Let's say sensor measurement is "door" ###

In [ ]:
belief = np.array([1./3, 1./3, 0, 0, 0, 0, 0, 0, 1./3, 0])

In [ ]:
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)

### Now suppose we get the following readings: ###
1. door
2. move right
3. door


In [ ]:
belief = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)

### Updating Belief
#### Since the state $x_t$ is unobservable, we maintain a belief about the state, $b(x_t)$, given the measurements. This process is also called filtering or state estimation. In mathematical terms, the belief is a probability distribution over all possible states, conditioned by the observations so far: 
> $b(x_t):= P(x_t∣e_{0:t})$ 
#### where $e_{0:t}$ is a short-hand notation for $(e_0,e_1,\cdots,e_t)$

#### We also define $\bar{b}(x_t)$ as the projected state or belief, which is the probability distribution over all the possible states at time $t$, given only past observations:
> $\bar{b}(x_t):= P(x_t∣e_{1:t-1})$ <br>

### Recursive estimation or Bayesian Filtering
#### The number of measurements we have to condition by in order to determine the belief increases unboundedly over time. 
#### This means: memory requirement as well as computation time increase unboundedly, since we have to consider all the measurements so far. 
#### Computationally tractable method: find a function $f$ such that 
> $b(x_{t+1})=f(b(x_t),e_{t+1})$
#### Two step process: (1) take the belief of the previous time step and project it to the new time step and (2) update it in accordance with new evidence. 

### Transition model
#### In real-world scenarios are stochastic: given past states $x_0, x_1, \cdots, x_t$, we cannot exactly tell what the state $x_{t+1}$ will be. However, we can assign likelihoods to each possible value for state $x_{t+1}$
> $P(x_{t+1}∣x_{0},x_1,\cdots,x_t)$
### Markov assumption
> $P(x_{t+1}∣x_{0},x_1,\cdots,x_t) = P(x_{t+1}∣x_t)$

### Sensor model
#### Environment may be only partially observable or the sensors may be inaccurate:
> $P(e_t∣x_t)$

## The Bayes Filter Algorithm

### Step 1: Projection

#### In this step, we project the belief of the previous time step to the current time step
> $\bar{b}(x_{t+1}) = \int_{x_t} P(x_{t+1}∣x_t)b(x_t)$

### Step 2: Update 
#### In this step, we update the projected belief with the new measurements $e_{t+1}$
> $b(x_{t+1}) = \eta P(e_{t+1}∣x_{t+1})\bar{b}(x_{t+1})$
#### Here, $\eta$ has the function of a normalizing constant (so we do not have to calculate it directly from its definition)

## Continuous Bayes filter
> $\bar{b}(x_{t+1})=\int_{x_t}P(x_{t+1}∣x_t)b(x_t)$ <br>
$b(x_{t+1})=\eta P(e_{t+1}∣x_{t+1})\bar{b}(x_{t+1})$

## Discrete Bayes filter
> $\bar{b}(x_{t+1})=\sum_{x_t}P(x_{t+1}∣x_t)b(x_t)$ <br>
$b(x_{t+1})=\eta P(e_{t+1}∣x_{t+1})\bar{b}(x_{t+1})$

<tr>
<td> <img src="wheel.png" alt="Drawing" style="width: 350px;"/> </td>
<td> <img src="roger.jpg" alt="Drawing" style="width: 350px;"/> </td>
</tr>

In [ ]:
hallway = np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0])

### Transition model:
#### The robot moves to the right one step eact time unit. But occassionally it does not move at all or moves two steps in a time unit.
> $P(x_{t+1} = i+2 \, \mbox{mod} \, 10 \mid x_{t} = i) = 0.05$ <br>
$P(x_{t+1}= i+1 \, \mbox{mod} \, 10 \mid x_{t} = i) = 0.9$ <br>
$P(x_{t+1}= i \mid x_{t}=i) = 0.05$

In [ ]:
trans_mat = np.zeros(shape=(10,10))
print(trans_mat)
print(" ")
for i in range(0,10):
    trans_mat[i,i] = 0.05
    trans_mat[i,(i+1)%10] = 0.9
    trans_mat[i,(i+2)%10] = 0.05
print(trans_mat)

### Sensor model:
#### The sensor correctly detects a door (reading of 1) or a wall (reading of 0) in 90% of the cases and in 10% of the cases it is incorrect
> $P(e_t = 1 \mid x_t=0,1,8)=0.9$ <br>
$P(e_t = 0 \mid x_t=0,1,8)=0.1$ <br>

>$P(e_t = 0 \mid x_t=2,3,4,5,6,7,9)=0.9$ <br>
$P(e_t = 1 \mid x_t=2,3,4,5,6,7,9)=0.1$ 

In [ ]:
sensor_mat = np.zeros(shape=(2,10))
sensor_mat[0,0] = 0.1  
sensor_mat[0,1] = 0.1
sensor_mat[0,2] = 0.9
sensor_mat[0,3] = 0.9  
sensor_mat[0,4] = 0.9
sensor_mat[0,5] = 0.9
sensor_mat[0,6] = 0.9  
sensor_mat[0,7] = 0.9 
sensor_mat[0,8] = 0.1
sensor_mat[0,9] = 0.9
sensor_mat[1,0] = 0.9  
sensor_mat[1,1] = 0.9
sensor_mat[1,2] = 0.1
sensor_mat[1,3] = 0.1  
sensor_mat[1,4] = 0.1
sensor_mat[1,5] = 0.1
sensor_mat[1,6] = 0.1  
sensor_mat[1,7] = 0.1 
sensor_mat[1,8] = 0.9
sensor_mat[1,9] = 0.1
print(sensor_mat)

#### Initial belief

In [ ]:
belief = np.array([.1, .1, .1, .1, .1, .1, .1, .1, .1, .1])

### Function: Projection
> $\bar{b}(x_{t+1})=\sum_{x_t}P(x_{t+1}∣x_t)b(x_t)$ 
#### Example:
> $\bar{b}(x_{t+1}=7)=\sum_{i=0}^{10}P(x_{t+1}=7∣x_t=i)b(x_t=i)$

In [ ]:
def projection(belief, trans_mat, project):
    n = len(belief)    
    for i in range(n):
        for j in range(n):
            project[i] += trans_mat[j][i]*belief[j]

In [ ]:
n = len(belief)
project = np.zeros(n)
projection(belief, trans_mat, project)
y_pos = np.arange(len(project))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(project), step=1))
plt.bar(y_pos, project)

#### Function: Update
> $b(x_{t+1})=\eta P(e_{t+1}∣x_{t+1})\bar{b}(x_{t+1})$
#### Example: sensor reading is 1:
> $b(x_{t+1})=\eta P(e_{t+1}=1∣x_{t+1})\bar{b}(x_{t+1})$

In [ ]:
def update(belief, project, sensor_mat, reading):
    n = len(belief)
    for i in range(n):
        belief[i] = sensor_mat[reading,i]*project[i]
    belief /= sum(belief)

In [ ]:
update(belief, project, sensor_mat, 1)
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)

#### Second sensor reading: 0

In [ ]:
n = len(belief)
project = np.zeros(n)
projection(belief, trans_mat, project)
update(belief, project, sensor_mat, 0)
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)

#### Third sensor reading: 0

In [ ]:
n = len(belief)
project = np.zeros(n)
projection(belief, trans_mat, project)
update(belief, project, sensor_mat, 0)
y_pos = np.arange(len(belief))
plt.ylim(0.0, 1.0)
plt.xticks(np.arange(0, len(belief), step=1))
plt.bar(y_pos, belief)